In [1]:
import pandas

In [2]:
usecols = ['borough_code', 'date_processed', 'house_flat',"ward_code",
       'inner_outer', 'local_authority', 'month',
       'post_code', 'post_code_clean', 'postcode', 'price', 'property_type',
       'quarter', 'tenure', 'town',
       'transaction_id', 'whether_new',
       'whether_newbuild', 'year', 'year_month']
london = pandas.read_csv("/home/vivek/Documents/mydata/iimb/london_real_estate/london_data_complete.csv",usecols=usecols,low_memory=False)

In [3]:
london["year_month_new"] = london["year"]*100 + london["month"]
london = london[london["year_month_new"]>=200901]

In [4]:
londonPrice = pandas.DataFrame(london.groupby(by=["post_code","year_month_new"])["price"].mean()).reset_index()

In [5]:
google_places = pandas.read_csv("/home/vivek/Documents/mydata/iimb/google_places_details/google_places_complete_with_missing.csv")
ndvi = pandas.read_csv("/home/vivek/Documents/mydata/iimb/satellite_images/ndvi.csv")
temp = pandas.read_csv("/home/vivek/Documents/mydata/iimb/economic_social/temperature.csv")
ndvi = ndvi.drop(["latitude","longitude"],axis=1)

In [6]:
londonPrice = londonPrice.merge(google_places,how="left",on="post_code")
londonPrice = londonPrice.merge(ndvi,how="left",on="post_code")
londonPrice = londonPrice.merge(temp,how="left",left_on="year_month_new",right_on="YearMon")

In [7]:
londonPrice.to_csv("/home/vivek/Documents/mydata/iimb/regression/regression_input.csv")

In [ ]:
## GMM Model

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import time

In [ ]:
londonPrice = pandas.read_csv("/home/vivek/Documents/mydata/iimb/regression/regression_input.csv")

In [ ]:
train = londonPrice[londonPrice["year_month_new"]<201401]
test = londonPrice[londonPrice["year_month_new"]>=201401]

In [ ]:
start_time = time.time()
print (start_time)
md = smf.mixedlm('price~year_month_new  + liquor_store_500 + airport_10000 + atm_500 + bank_500 + bus_station_500 + church_500 + hospital_500 + park_500 + parking_500 + pharmacy_500 + police_500 + restaurant_500 + school_500 + taxi_stand_2000 + train_station_1000 + university_500 + ndvi_max + ndvi_mean + ndvi_median + ndvi_min  + Month + Tmax + Tmin + af_days + rain_mm + sun_hours',train, groups=train["post_code"],re_formula="~year_month_new")
mdf = md.fit()
print(mdf.summary())
print("Total Time Taken Is %s seconds" % (time.time() - start_time))